In [1]:
import warnings
warnings.filterwarnings("ignore")

from xai_agg import *

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

import pandas as pd
import numpy as np

import dill

2025-01-14 11:57:12.996767: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-14 11:57:13.019951: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
raw_data = pd.read_csv("../data/Student_Depression_Dataset.csv")
display(raw_data)
display(raw_data["Profession"].value_counts())

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2,Male,33.0,Visakhapatnam,Student,5.0,0.0,8.97,2.0,0.0,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
1,8,Female,24.0,Bangalore,Student,2.0,0.0,5.90,5.0,0.0,5-6 hours,Moderate,BSc,No,3.0,2.0,Yes,0
2,26,Male,31.0,Srinagar,Student,3.0,0.0,7.03,5.0,0.0,Less than 5 hours,Healthy,BA,No,9.0,1.0,Yes,0
3,30,Female,28.0,Varanasi,Student,3.0,0.0,5.59,2.0,0.0,7-8 hours,Moderate,BCA,Yes,4.0,5.0,Yes,1
4,32,Female,25.0,Jaipur,Student,4.0,0.0,8.13,3.0,0.0,5-6 hours,Moderate,M.Tech,Yes,1.0,1.0,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27896,140685,Female,27.0,Surat,Student,5.0,0.0,5.75,5.0,0.0,5-6 hours,Unhealthy,Class 12,Yes,7.0,1.0,Yes,0
27897,140686,Male,27.0,Ludhiana,Student,2.0,0.0,9.40,3.0,0.0,Less than 5 hours,Healthy,MSc,No,0.0,3.0,Yes,0
27898,140689,Male,31.0,Faridabad,Student,3.0,0.0,6.61,4.0,0.0,5-6 hours,Unhealthy,MD,No,12.0,2.0,No,0
27899,140690,Female,18.0,Ludhiana,Student,5.0,0.0,6.88,2.0,0.0,Less than 5 hours,Healthy,Class 12,Yes,10.0,5.0,No,1


Profession
Student                   27870
Architect                     8
Teacher                       6
Digital Marketer              3
Content Writer                2
Chef                          2
Doctor                        2
Pharmacist                    2
Civil Engineer                1
UX/UI Designer                1
Educational Consultant        1
Manager                       1
Lawyer                        1
Entrepreneur                  1
Name: count, dtype: int64

In [3]:
preprocessed_data = raw_data.drop(columns=["id", "City", "Profession", "Degree"])

# Treat column names
preprocessed_data.rename(columns={
    'Have you ever had suicidal thoughts ?': 'Suicidal',
    "Family History of Mental Illness": "Family_History",
    "Gender": "GenderMale"
}, inplace=True)
preprocessed_data.columns = preprocessed_data.columns.str.replace(' ', '_')
preprocessed_data.columns = preprocessed_data.columns.str.replace('/', '_')

# Encoding categorical variables
categorical_features = []
preprocessed_data["GenderMale"] = preprocessed_data["GenderMale"].map({"Male": 1, "Female": 0})
preprocessed_data["Dietary_Habits"] = preprocessed_data["Dietary_Habits"].map({"Others": 0, "Unhealthy": 1, "Moderate": 2, "Healthy": 3})
preprocessed_data["Suicidal"] = preprocessed_data["Suicidal"].map({"Yes": 1, "No": 0})
preprocessed_data["Family_History"] = preprocessed_data["Family_History"].map({"Yes": 1, "No": 0})
preprocessed_data["Sleep_Duration"] = preprocessed_data["Sleep_Duration"].map({"Others": 0, "Less than 5 hours": 1, "5-6 hours": 2, "7-8 hours": 3, "More than 8 hours": 4})

preprocessed_data, _ = train_test_split(preprocessed_data, test_size=0.875, stratify=preprocessed_data['Depression'], random_state=42)
preprocessed_data.dropna(inplace=True)

display(preprocessed_data)

,GenderMale,Age,Academic_Pressure,Work_Pressure,CGPA,Study_Satisfaction,Job_Satisfaction,Sleep_Duration,Dietary_Habits,Suicidal,Work_Study_Hours,Financial_Stress,Family_History,Depression
22551,0,33.0,3.0,0.0,5.86,5.0,0.0,2,1,1,8.0,1.0,1,0
15512,1,32.0,3.0,0.0,6.16,4.0,0.0,4,2,0,0.0,3.0,0,0
21944,1,19.0,3.0,0.0,7.80,4.0,0.0,4,2,0,2.0,1.0,0,0
15462,1,32.0,4.0,0.0,6.89,3.0,0.0,2,2,1,5.0,1.0,0,1
21760,0,33.0,2.0,0.0,8.17,2.0,0.0,3,3,0,6.0,4.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10927,1,22.0,5.0,0.0,5.27,1.0,0.0,4,1,0,9.0,5.0,0,1
12611,0,23.0,5.0,0.0,6.27,1.0,0.0,1,1,0,1.0,4.0,1,1
14848,1,21.0,1.0,0.0,8.59,1.0,0.0,3,2,0,3.0,2.0,1,0
13476,0,29.0,1.0,0.0,9.71,3.0,0.0,4,2,0,6.0,4.0,0,0


In [4]:
y = preprocessed_data["Depression"]
X = preprocessed_data.drop(columns=["Depression"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"ROC AUC: {roc_auc_score(y_test, y_pred)}")

Accuracy: 0.8151862464183381
ROC AUC: 0.8017150225660865


# Experiments
# RAE-T vs. RAE-E | 10 samples
### Execution

In [6]:
results, metadata = evaluate_aggregate_explainer(
    clf, X_train, X_test, categorical_features,
    metrics_sets=[['nrc', 'sensitivity_spearman', 'faithfulness_corr']],
    mcdm_algs=[pymcdm.methods.TOPSIS(), pymcdm.methods.EDAS()],
    n_instances=10
)

metadata["description"] = "RAE-T vs RAE-S, 10 samples"

with open('pickles/student_depression/RAE-T_vs_RAE-S_10.pkl', 'wb') as f:
    dill.dump(ExperimentRun(metadata, results), f)

Selected indexes: [ 4888 10193 25349 13937 17693 17325 27257 17960 27040 15147]
Epoch 1/500
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.1088 - val_loss: 1.0659
Epoch 2/500
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 619us/step - loss: 1.0444 - val_loss: 1.0100
Epoch 3/500
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 614us/step - loss: 0.9912 - val_loss: 0.9575
Epoch 4/500
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 653us/step - loss: 0.9569 - val_loss: 0.9086
Epoch 5/500
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 647us/step - loss: 0.9004 - val_loss: 0.8666
Epoch 6/500
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 616us/step - loss: 0.8562 - val_loss: 0.8327
Epoch 7/500
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 592us/step - loss: 0.8208 - val_loss: 0.8064
Epoch 8/500
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 646us/step - loss: 0.8002 - val_loss: 0.7870
Epoch 9/500
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 617us/step - loss: 0.7946 - val_loss: 0.7723
Epoch 10/500
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 568us/step - loss: 0.7664 - val_loss: 0.7611
Epoch 11/500
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 718us/st

### Analysis

In [7]:
with open('pickles/student_depression/RAE-T_vs_RAE-S_10.pkl', 'rb') as f:
    exp = dill.load(f)

In [8]:
methods = ["RAE-T", "RAE-E"]
for i, method in enumerate(methods):
    print(f"{method}:\n")
    display(exp.results[i])
    wca = count_worst_case_avoidances(exp.results[i], [False, True, True], 1)
    print(f"Worst case avoidances:\n\t- for all metrics: {wca[0]}\n\t- for 2/3 metrics: {wca[1]}")
    print("AVG:")
    display(get_expconfig_mean_results(exp, i))
    print("\n")

RAE-T:



[                              nrc  sensitivity_spearman  faithfulness_corr
 LimeWrapper             20.629599              0.956044           0.303474
 ShapTabularTreeWrapper  18.124362              0.961433           0.182400
 AnchorWrapper           16.960760              0.502515           0.728873
 AggregateExplainer      20.687438              0.968132           0.197572,
                               nrc  sensitivity_spearman  faithfulness_corr
 LimeWrapper             22.002276              0.977473           0.377080
 ShapTabularTreeWrapper  21.704769              0.999449           0.190546
 AnchorWrapper           17.559590              0.918255           0.997531
 AggregateExplainer      20.798359              0.988462           0.911884,
                               nrc  sensitivity_spearman  faithfulness_corr
 LimeWrapper             21.582348              0.975824           0.709159
 ShapTabularTreeWrapper  21.582348              0.997796           0.828122
 AnchorWra

Worst case avoidances:
	- for all metrics: 6
	- for 2/3 metrics: 9
AVG:


,nrc,sensitivity_spearman,faithfulness_corr
AggregateExplainer,21.214399,0.976166,0.605186
AnchorWrapper,17.796283,0.804567,0.532620
LimeWrapper,22.295143,0.971978,0.640042
ShapTabularTreeWrapper,20.866798,0.991295,0.669723




RAE-E:



[                              nrc  sensitivity_spearman  faithfulness_corr
 LimeWrapper             21.218686              0.971978           0.010488
 ShapTabularTreeWrapper  18.124362              0.946006           0.195487
 AnchorWrapper           16.960760              0.668886           0.289283
 AggregateExplainer      21.282497              0.953846           0.508951,
                               nrc  sensitivity_spearman  faithfulness_corr
 LimeWrapper             19.289516              0.978571           0.701283
 ShapTabularTreeWrapper  21.704769              0.996694           0.737929
 AnchorWrapper           17.559590              0.880556           0.907452
 AggregateExplainer      18.913895              0.965934           0.882225,
                               nrc  sensitivity_spearman  faithfulness_corr
 LimeWrapper             20.340532              0.965385           0.925581
 ShapTabularTreeWrapper  21.582348              0.995041           0.157819
 AnchorWra

Worst case avoidances:
	- for all metrics: 3
	- for 2/3 metrics: 9
AVG:


,nrc,sensitivity_spearman,faithfulness_corr
AggregateExplainer,21.530867,0.952974,0.630865
AnchorWrapper,17.601674,0.828051,0.574866
LimeWrapper,20.074849,0.967802,0.614056
ShapTabularTreeWrapper,20.866798,0.990413,0.472897
